## <center>Animation with  Plotly 3.0.0`<br>Rotation about z-axis implemented as a rotation of the camera eye

In order to animate the rotation of a 3d object with Plotly it is much more convenient to rotate the camera eye about that object. We define below  such
a rotation. 

In [1]:
import numpy as np
from numpy import sin, cos
from plotly.graph_objs import FigureWidget
import plotly.graph_objs as go

The rotation about Oz, with $\theta$ radians, is defined 
 via a planar rotation in each plane z=cst, considered as a rotation in the complex plane:
 $$rot_\theta(x+j y)=e^{ j \theta}(x+j y)$$

In [2]:
def rotate_z(x, y, z, theta):
    w=x+1j*y
    return np.real(np.exp(2*np.pi*1j*theta)*w), np.imag(np.exp(2*np.pi*1j*theta)*w), z

Define the surface:

In [3]:
u=np.linspace(0, 2*np.pi, 75)
v=np.linspace(0, 2*np.pi, 75)
u,v=np.meshgrid(u,v)

x=cos(v)*(6-(5/4+sin(3*u))*sin(u-3*v))
y=sin(v)*(6-(5/4+sin(3*u))*sin(u-3*v))
z=-cos(u-3*v)*(5/4+sin(3*u))

In [4]:
balance=[[0.0, 'rgb(23, 28, 66)'],
         [0.1, 'rgb(41, 61, 150)'],
         [0.2, 'rgb(21, 112, 187)'],
         [0.3, 'rgb(89, 155, 186)'],
         [0.4, 'rgb(169, 194, 202)'],
         [0.5, 'rgb(240, 236, 235)'],
         [0.6, 'rgb(219, 177, 163)'],
         [0.7, 'rgb(201, 118, 90)'],
         [0.8, 'rgb(179, 56, 38)'],
         [0.9, 'rgb(125, 13, 41)'],
         [1.0, 'rgb(60, 9, 17)']]

In [5]:
axes_style = dict(showbackground=True, 
                  backgroundcolor="rgb(230, 230,230)",
                  gridcolor="rgb(255, 255, 255)",      
                  zerolinecolor="rgb(255, 255, 255)")

In [6]:
tr=go.Surface(x=x,
           y=y,
           z=z, 
           colorscale=balance,
           showscale=False,   
           #colorbar=dict(thickness=20, ticklen=4, len=0.6),
           lighting=dict(ambient=0.5,
                         diffuse=1,
                         fresnel=4,
                                    
                         specular=0.5,
                         roughness=0.5
                        ),
                 lightposition=dict(x=100,
                                    y=100,
                                    z=1000)
          )

x_eye, y_eye, z_eye=1.25,1.25,0.8                             
layout=dict(title='Camera eye rotation',
            width=800,
            height=800, 
            autosize=False,
            scene=dict(camera=dict(eye=dict(x=x_eye, y=y_eye, z=z_eye)),
                       xaxis=axes_style,
                       yaxis=axes_style,
                       zaxis=axes_style,
                       aspectratio=dict(x=1,
                                        y=1,
                                        z=0.35)
                                        )                        
                                     )
                                      

In [7]:
fw=go.FigureWidget(data=[tr], layout=layout)

Animate the rotation of the camera eye about Oz:

In [8]:
import ipywidgets as iw
slider = iw.IntSlider(value=0, min=0, max=180, step=2, description='rotation angle')
slider.layout=dict(margin='100px 80px 40px 5px', width='500px')#margin describes top, right, bottom, left

In [9]:
def angle_changed(change):
    theta = np.pi*slider.value/180
    xe, ye, ze=rotate_z(x_eye, y_eye, z_eye, -theta)
    fw.layout.scene.camera.eye=dict(x=xe, y=ye, z=ze)

slider.observe(angle_changed, 'value')

In [10]:
play_button = iw.Play(value=0, min=0, max=180, step=2, description="Play")
play_button.layout=dict(margin='100px 10px 50px 100px')
iw.link((play_button, 'value'), (slider, 'value'))
iw.VBox([iw.HBox([play_button, slider]), fw])